## Detection of roadside vegetation

## Imports

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from modules.ml_utils import * 
from modules.image_utils import *
from modules.utils import *
from modules.image_sample import ImageSample
from find_green import simple_find_green
from modules.preprocess import Preprocess
%matplotlib inline

In [2]:
cv2.destroyAllWindows()

## Simple 'find green' method

In [7]:
file_path = '/home/marko/ssip/pysolution/denis_rsz.jpg'
img = cv2.imread(file_path)
result = simple_find_green(file_path)
res = np.hstack((img,binarized_to_rgb(result)))
cv2_imshow('res', res)


In [9]:
# CLAHE demo
img = cv2.imread('/home/marko/ssip/pysolution/denis_rsz.jpg')
normalized = apply_clahe_rgb(img)
res = np.hstack((img,normalized))
cv2_imshow('res',res)

# Train model

In [10]:
# initialize directories
root_dir = './data/textures'
positive_textures = os.path.join(root_dir, 'positive')
negative_textures = os.path.join(root_dir, 'negative')

negative_files = list_files(negative_textures)
positive_files = list_files(positive_textures)


In [11]:
# load textures
samples = []

def load_labeles_samples(files, label): 
    return [(ImageSample(path=img_file), label) for img_file in files]

samples = []
samples.extend(load_labeles_samples(negative_files, 0))
samples.extend(load_labeles_samples(positive_files, 1))
    

In [12]:
# load samples
X = []
y = []
samples_per_texture = 1024

for img_sample, label in samples:
    X.extend(img_sample.to_samples(samples_per_texture))
    y.extend([label] * samples_per_texture)

In [14]:
# sanity check
print '# Sample images', len(samples)
print '# samples', len(X)
print '# labels', len(y)
print '# features', len(X[0])

# Sample images 161
# samples 164864
# labels 164864
# features 160


### Try different models

In [15]:
models = [DummyClassifier(), GaussianNB(), LinearSVC(), SVC(), DecisionTreeClassifier(), RandomForestClassifier()]
parameters = [{}, {}, {},{'kernel':('linear', 'rbf','poly','sigmoid')},{}, {}]

models = [GaussianNB()]
parameters = [{}]

In [16]:
evaluate_models(X, y, models, parameters, verbose=True)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    2.1s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.7s finished


Model GaussianNB()
Model best_params: {}
Model score : 0.730226445613
Confusion matrix:
 [[20197  3718]
 [ 9625 15920]]
binary scores:
	 P  = 0.677251693381
	 R  = 0.84453272005
	 F1 = 0.751698085118


### Single model training

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = RandomForestClassifier()
model.fit(X_train, y_train)
printScore(y_train, model.predict(X_train))
printScore(y_test, model.predict(X_test))

micro scores:
	 P  = 0.997903018959
	 R  = 0.997903018959
	 F1 = 0.997903018959
micro scores:
	 P  = 0.942842701173
	 R  = 0.942842701173
	 F1 = 0.942842701173


In [16]:
# Free data from textures
X = []
y = []
X_train = X_test = y_train = y_test = []

In [17]:
# Store model
filename = './stored_models/tree_clahe.joblib.pkl'
store_model(filename, model)

In [3]:
# Load saved model
filename = './stored_models/tree2.joblib.pkl'
model = load_model(filename)

In [4]:
@time_usage
def analyze_single_image(file_path, model):
    print 'Processing file', file_path
    img = cv2.imread(file_path)
    #img = cv2_resize(img, y=600)

    img_name = os.path.basename(file_path).split('.')[0]
    #file_path = './' + img_name + '_rsz.jpg'
    #cv2.imwrite(file_path, img)


    out_path = './' + img_name + '_out.jpg'
    img = ImageSample(path=file_path)
    #raw_input('continue')
    result = img.find_vegetation(model)
    simple = simple_find_green(file_path)

    simple[simple==1] = 255
    result[result==1] = 255
    result = binarized_to_rgb(result)
    simple = binarized_to_rgb(simple)


    grid = np.hstack([img.img_rgb, result, simple])
    cv2.imwrite(out_path, grid)
    
def analyze_whole_dir(dir_path, model):
    for file_path in  list_files(dir_path):
        analyze_single_image(file_path, model)

In [8]:
# single image
file_path = '/home/marko/PycharmProjects/SSIP16/data/ssip_data/test.jpg'
analyze_single_image(file_path, model)

Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test.jpg


In [5]:
# whole directory analysis
dir_path = '/home/marko/PycharmProjects/SSIP16/data/ssip_data/'
analyze_whole_dir(dir_path, model)

Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test8.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test10.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test15.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test11.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test3.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test1.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test13.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test2.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test12.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test16.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test9.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test7.jpg
Processing file /home/marko/PycharmProjects/SSIP16/data/ssip_data/test0.jpg
Proces

In [28]:
cv2.destroyAllWindows()


# Texture annotator

slides rect across photo and user types 1 for positive sample or 0 for negative sample

In [10]:
list_files = lambda path: [os.path.join(path, img_path) for img_path in os.listdir(path)]
root = './data/images'
pos_dir = './new_textures/pos/'
neg_dir = './new_textures/neg/'
map(init_dir, [root, pos_dir, neg_dir])


rect_size = 150
step = 15
current_id  = 0

for file_path in list_files(root):
    img = cv2.imread(file_path)
    height, width, _ = img.shape
    print file_path, height, width
    i, j = (0, 0)
    
    def get_file_name(dir_path, i):
        return os.path.join(dir_path, '%d.jpg' % i)
    
    def slide_rect():
        global current_id, i, j
        def out_of_range(i, j):
            return i < 0 or i >= (height - rect_size) \
                    or j < 0 or j >= (width - rect_size)
        while True:
 
                tmp = img.copy()
                texture = img[i:i+rect_size, j:j+rect_size,]
                cv2.rectangle(tmp,(j, i),(j+rect_size,i+rect_size),(0,255,0),3)
                

                cv2.imshow('original', tmp)
                cv2.imshow('texture', texture)
                key = cv2.waitKey(0) 
                
                di = 0
                dj = 0
                if key == ord('n'): 
                    print 'continue'; return
                elif key == ord('q'): 
                    return True
                #store texture
                elif key == ord('1'):
                    current_id += 1; 
                    cv2.imwrite(get_file_name(pos_dir, current_id), texture)
                elif key == ord('2'):
                    current_id += 1; 
                    cv2.imwrite(get_file_name(neg_dir, current_id), texture)
                # movement
                elif key == ord('w'): di = -step
                elif key == ord('s'): di = step
                elif key == ord('a'): dj = -step
                elif key == ord('d'): dj = step
                    
                ni, nj = (i+di, j+dj)
                if not out_of_range(ni, nj):
                    i, j = (ni, nj)
                    
                    
    ret = slide_rect()
    if ret: break
print 'Completed'

./data/images/v2.jpg 1800 2405
Completed


In [11]:
cv2.destroyAllWindows()